In [1]:
pip install fasttext

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import fasttext
import bz2
import csv
from sklearn.metrics import roc_auc_score
import os

In [4]:
DIR='/Users/anastasiamoiseva/Downloads'

In [135]:
data = bz2.BZ2File(DIR+"/amazonreviews/train.ft.txt.bz2")
data = data.readlines()
data = [x.decode('utf-8') for x in data]
print(len(data))



3600000


In [6]:
data = pd.DataFrame(data)
data.to_csv("train.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [7]:
model = fasttext.train_supervised('train.txt',label_prefix='__label__', thread=4, epoch = 10)
print(model.labels, 'are the labels or targets the model is predicting')

['__label__1', '__label__2'] are the labels or targets the model is predicting


In [9]:
test = bz2.BZ2File(DIR+"/amazonreviews/test.ft.txt.bz2")
test = test.readlines()
test = [x.decode('utf-8') for x in test]
print(len(test), 'number of records in the test set') 

400000 number of records in the test set


In [10]:
new = [w.replace('__label__2 ', '') for w in test]
new = [w.replace('__label__1 ', '') for w in new]
new = [w.replace('\n', '') for w in new]

In [14]:
labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test]
pred_labels = [0 if x == ['__label__1'] else 1 for x in pred[0]]

In [11]:
pred = model.predict(new)

In [12]:
print(pred[0][0], 'is the predicted label')
print(pred[0][1], 'is the probability score')

['__label__2'] is the predicted label
['__label__2'] is the probability score


In [15]:
print(roc_auc_score(labels, pred_labels))

0.91719


In [136]:
train_labels  = [0 if x.split(' ')[0] == '__label__1' else 1 for x in data]

In [137]:
new_train = [w.replace('__label__2 ', '') for w in data]
new_train = [w.replace('__label__1 ', '') for w in new]
new_train = [w.replace('\n', '') for w in new]

In [151]:
labels_tr = pd.DataFrame(train_labels)

In [160]:
data_train = pd.DataFrame(new_train, dtype=str)
data_train[:4]

,0
0,Great CD: My lovely Pat has one of the GREAT v...
1,One of the best game music soundtracks - for a...
2,Batteries died within a year ...: I bought thi...
3,"works fine, but Maha Energy is better: Check o..."


In [186]:
data_train.columns = ['Message', 'Labels']

In [195]:
data_train.tail()


,Message,Labels
399995,Unbelievable- In a Bad Way: We bought this Tho...,1
399996,"Almost Great, Until it Broke...: My son reciev...",0
399997,Disappointed !!!: I bought this toy for my son...,1
399998,Classic Jessica Mitford: This is a compilation...,0
399999,"Comedy Scene, and Not Heard: This DVD will be ...",1


In [194]:
data_train.shape

(400000, 2)

In [177]:
data_train['Labels'] = labels_tr

In [178]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix

In [198]:
bow_transformer = CountVectorizer(lowercase=False).fit(data_train['Message'])
print(len(bow_transformer.vocabulary_))

340716


In [199]:
amazon_bow = bow_transformer.transform(data_train['Message'])

In [200]:
data_train.shape


(400000, 2)

In [201]:
print('Shape of Sparse Matrix: ',amazon_bow.shape)
print('Amount of non-zero occurences:',amazon_bow.nnz)

Shape of Sparse Matrix:  (400000, 340716)
Amount of non-zero occurences: 22760250


In [202]:
sparsity =(100.0 *amazon_bow.nnz/(amazon_bow.shape['Message']*amazon_bow.shape[1]))
print('sparsity:{}'.format(round(sparsity)))

TypeError: tuple indices must be integers or slices, not str

In [203]:
message4=data_train['Message'][3]
print(message4)

works fine, but Maha Energy is better: Check out Maha Energy's website. Their Powerex MH-C204F charger works in 100 minutes for rapid charge, with option for slower charge (better for batteries). And they have 2200 mAh batteries.


In [204]:
bow4=bow_transformer.transform([message4])
print(bow4)
print(bow4.shape)

  (0, 479)	1
  (0, 3661)	1
  (0, 16499)	1
  (0, 31038)	1
  (0, 37478)	1
  (0, 56961)	2
  (0, 98470)	1
  (0, 100511)	2
  (0, 124559)	1
  (0, 157555)	1
  (0, 188972)	2
  (0, 190640)	2
  (0, 195657)	1
  (0, 199525)	2
  (0, 199533)	1
  (0, 230567)	1
  (0, 232401)	3
  (0, 241405)	1
  (0, 247481)	1
  (0, 252126)	1
  (0, 261087)	1
  (0, 266172)	1
  (0, 275856)	1
  (0, 276807)	1
  (0, 291535)	1
  (0, 307782)	1
  (0, 321364)	1
  (0, 335358)	1
  (0, 337360)	1
  (0, 338249)	2
(1, 340716)


In [205]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer().fit(amazon_bow)
tfidf4 = tfidf_transformer.transform(bow4)
print(tfidf4)

  (0, 338249)	0.1705579646334395
  (0, 337360)	0.03976565582701023
  (0, 335358)	0.12334808179757155
  (0, 321364)	0.057069492687200525
  (0, 307782)	0.15681912609077459
  (0, 291535)	0.17915094760195382
  (0, 276807)	0.05489020546153735
  (0, 275856)	0.1293923729376425
  (0, 266172)	0.09854872705107565
  (0, 261087)	0.21223183120828037
  (0, 252126)	0.02933826102716621
  (0, 247481)	0.03408429495220594
  (0, 241405)	0.04163513783867195
  (0, 232401)	0.1024374830521529
  (0, 230567)	0.09410370707694916
  (0, 199533)	0.13824837119640393
  (0, 199525)	0.2416619093286253
  (0, 195657)	0.04037133767437156
  (0, 190640)	0.1370243478657787
  (0, 188972)	0.23812091241481786
  (0, 157555)	0.13090757275492307
  (0, 124559)	0.24114221884528292
  (0, 100511)	0.4692905183070882
  (0, 98470)	0.21417630959208234
  (0, 56961)	0.37882071606953455
  (0, 37478)	0.1411995194245566
  (0, 31038)	0.26942481598567664
  (0, 16499)	0.08429141354688773
  (0, 3661)	0.21045666152495238
  (0, 479)	0.11451825536394

In [206]:
amazon_tfidf=tfidf_transformer.transform(amazon_bow)
print(amazon_tfidf.shape)

(400000, 340716)


In [209]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(amazon_tfidf,data_train['Labels'])

In [210]:
all_predictions = spam_detect_model.predict(amazon_tfidf)
print(all_predictions)

[1 0 1 ... 0 1 1]


In [212]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(data_train['Labels'],all_predictions))
print(confusion_matrix(data_train['Labels'],all_predictions))

              precision    recall  f1-score   support

           0       0.78      0.62      0.69    197932
           1       0.69      0.83      0.75    202068

    accuracy                           0.73    400000
   macro avg       0.74      0.73      0.72    400000
weighted avg       0.74      0.73      0.72    400000

[[123628  74304]
 [ 34523 167545]]


In [157]:
pipeline.fit(data_train,labels_tr)

TypeError: expected string or bytes-like object